# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

In [3]:
df.size

283660

## Treat naming

In [4]:
def format_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.rename(columns = {'employmentstatus': 'employment_status'})
    return df

def print_unique(df):
    for column in ['response', 'coverage', 'education', 'employment_status', 'gender', 'location_code', 'marital_status',
                   'policy', 'renew_offer_type', 'sales_channel', 'vehicle_class', 'vehicle_size', 'vehicle_type']:
        print(f'{column}: {df[column].unique()}')

def replace(df):
    return df.replace({
    'vehicle_size' : {'Medsize': 'Mid'},
    'vehicle_class': {'Luxury SUV': 'Luxury', 'Luxury Car': 'Luxury', 'SUV': 'Luxury'}})

In [5]:
df = format_columns(df)
df = replace(df)
df.drop('unnamed:_0', axis = 1, inplace = True)

In [6]:
print_unique(df) # all okay except vehicle_class
df.head(2)

response: ['No' 'Yes' nan]
coverage: ['Basic' 'Extended' 'Premium']
education: ['College' 'Bachelor' 'High School or Below' 'Doctor' 'Master']
employment_status: ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
gender: ['M' 'F']
location_code: ['Suburban' 'Urban' 'Rural']
marital_status: ['Married' 'Single' 'Divorced']
policy: ['Corporate L3' 'Personal L3' 'Personal L2' 'Corporate L2' 'Personal L1'
 'Special L1' 'Corporate L1' 'Special L3' 'Special L2']
renew_offer_type: ['Offer3' 'Offer4' 'Offer2' 'Offer1']
sales_channel: ['Agent' 'Call Center' 'Branch' 'Web']
vehicle_class: ['Four-Door Car' 'Luxury' 'Two-Door Car' 'Sports Car' nan]
vehicle_size: ['Mid' 'Small' 'Large' nan]
vehicle_type: [nan 'A']


,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Mid,NaN
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Mid,NaN


## Treat NaN values

In [7]:
print(df.isnull().sum())

customer                            0
state                             631
customer_lifetime_value             0
response                          631
coverage                            0
education                           0
effective_to_date                   0
employment_status                   0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto                0
months_since_last_claim           633
months_since_policy_inception       0
number_of_open_complaints         633
number_of_policies                  0
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                  0
vehicle_class                     622
vehicle_size                      622
vehicle_type                     5482
dtype: int64


In [8]:
# drop Nan responses since these are just a few rows compared to the size of the dataset and I dont want skew the dataframe for exercise 1
# the state Nan also disappear by doing that, it seems that there is generally data missing in these rows
#df.dropna(subset = 'response', inplace = True) 

# drop these two rows aswell because the left columns with Nan values all have suspiciously the same sum
#df.dropna(subset = 'vehicle_class', inplace = True)
#df.dropna(subset = 'months_since_last_claim', inplace = True)
#print(df.isnull().sum())

In [9]:
# use mode for the vehicle type columns containing NaN values since the sum of missing values is not much compared to over 200,000 customers
def fill_nan(df):
    for column in ['state', 'response', 'vehicle_class', 'months_since_last_claim', 'vehicle_size', 
                   'number_of_open_complaints', 'vehicle_type']:
        df[column] = df[column].fillna(df[column].mode()[0])
    return df

In [10]:
df = fill_nan(df)
print(df.isnull().sum())

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employment_status                0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
vehicle_type                     0
dtype: int64


## Convert floats to integers

In [11]:
df.dtypes
# customer_lifetime_value, number_of_open_complaints, total_claim_amount to integer

customer                          object
state                             object
customer_lifetime_value          float64
response                          object
coverage                          object
education                         object
effective_to_date                 object
employment_status                 object
gender                            object
income                             int64
location_code                     object
marital_status                    object
monthly_premium_auto               int64
months_since_last_claim          float64
months_since_policy_inception      int64
number_of_open_complaints        float64
number_of_policies                 int64
policy_type                       object
policy                            object
renew_offer_type                  object
sales_channel                     object
total_claim_amount               float64
vehicle_class                     object
vehicle_size                      object
vehicle_type    

In [12]:
def float_to_integers(df):
    df['total_claim_amount'] = df['total_claim_amount'].apply(int)
    df['customer_lifetime_value'] = df['customer_lifetime_value'].apply(int)
    df['months_since_last_claim'] = df['months_since_last_claim'].apply(int)
    df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(int)
    return df

In [13]:
df = float_to_integers(df)
df.dtypes

customer                         object
state                            object
customer_lifetime_value           int64
response                         object
coverage                         object
education                        object
effective_to_date                object
employment_status                object
gender                           object
income                            int64
location_code                    object
marital_status                   object
monthly_premium_auto              int64
months_since_last_claim           int64
months_since_policy_inception     int64
number_of_open_complaints         int64
number_of_policies                int64
policy_type                      object
policy                           object
renew_offer_type                 object
sales_channel                    object
total_claim_amount                int64
vehicle_class                    object
vehicle_size                     object
vehicle_type                     object


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [14]:
new_df1 = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
new_df1.head(5)

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,California,11009,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358,Luxury,Mid,A
236,YJ16163,Oregon,11009,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0,1,Special Auto,Special L3,Offer2,Agent,1358,Luxury,Mid,A
419,GW43195,Oregon,25807,Yes,Extended,College,2/13/11,Employed,F,71210,...,1,2,Personal Auto,Personal L2,Offer1,Branch,1027,Luxury,Small,A
442,IP94270,Arizona,13736,Yes,Premium,Master,2/13/11,Disabled,F,16181,...,0,8,Personal Auto,Personal L2,Offer1,Web,1261,Luxury,Mid,A
587,FJ28407,California,5619,Yes,Premium,High School or Below,1/26/11,Unemployed,M,0,...,0,1,Personal Auto,Personal L1,Offer2,Web,1027,Luxury,Mid,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [15]:
new_df2 = df[(df['response'] == 'Yes')]
new_df2.groupby(['policy_type', 'gender'])['total_claim_amount'].mean()
# On average, all policy types have equal claim amounts.
# Females and males don't have a huge difference in claim amounts though females seem 
# to claim a bit more in the coporate and special auto policies, on average.

policy_type     gender
Corporate Auto  F         433.295858
                M         408.123377
Personal Auto   F         452.498148
                M         456.576493
Special Auto    F         452.857143
                M         429.125000
Name: total_claim_amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [20]:
grouped = df.groupby('state')
grouped.size() # all states have more than 500 customers

state
Arizona       1937
California    4183
Nevada         993
Oregon        2909
Washington     888
dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [25]:
max_total_revenue = df.groupby(['education', 'gender'])['customer_lifetime_value'].max()
min_total_revenue = df.groupby(['education', 'gender'])['customer_lifetime_value'].min()
med_total_revenue = df.groupby(['education', 'gender'])['customer_lifetime_value'].median()

print(max_total_revenue) 
# The maximum total revenues range between 50,000 and 80,000$.
# The maximum female total revenue is slightly higher or almost equal for all education level except High School or Below.
# The maxmium male total revenue is 30,000$ higher for this education.
print('\n')
print(min_total_revenue)
# The minimum total revenues range around 2,000$.
# The male and female total revenues are almost equal.
print('\n')
print(med_total_revenue)
# The median total revenues range between 5,000 and 6,000$.
# The male and female medians are very similar while the female total revenue is higher for customers with a Bachelors and Masters degree. 

education             gender
Bachelor              F         73225
                      M         67907
College               F         61850
                      M         61134
Doctor                F         44856
                      M         32677
High School or Below  F         55277
                      M         83325
Master                F         51016
                      M         50568
Name: customer_lifetime_value, dtype: int64


education             gender
Bachelor              F         1904
                      M         1898
College               F         1898
                      M         1918
Doctor                F         2395
                      M         2267
High School or Below  F         2144
                      M         1940
Master                F         2417
                      M         2272
Name: customer_lifetime_value, dtype: int64


education             gender
Bachelor              F         5640.0
                      M         

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here